In [ ]:
# 安装 paddle-gpu 2.3.0，需要CUDA 11.2+
!python -m pip install paddlepaddle-gpu==2.3.0.post112 -f https://www.paddlepaddle.org.cn/whl/windows/mkl/avx/stable.html

In [ ]:
# 正式开始实验之前首先通过如下命令安装最新版本的 paddlenlp
!pip install --upgrade paddlenlp

In [2]:
%cd ./work/

/home/aistudio/work


In [ ]:
!$unset CUDA_VISIBLE_DEVICES
!python -u -m paddle.distributed.launch --gpus "0" train.py \
       --train_set /home/aistudio/train-with-afqmc.txt \
       --dev_set /home/aistudio/dev-with-afqmc.txt \
       --device gpu \
       --eval_step 400 \
       --save_step 400 \
       --save_dir /home/aistudio/model \
       --train_batch_size 128 \
       --learning_rate 2E-5 \
       --rdrop_coef 0.0 \
       --seed 2022 \
       --epochs 3 
       # --init_from_ckpt /home/aistudio/model/model_400/model_state.pdparams

In [ ]:
!$unset CUDA_VISIBLE_DEVICES
!python -u -m paddle.distributed.launch --gpus "0" train.py \
       --train_set /home/aistudio/dev-with-afqmc.txt \
       --dev_set /home/aistudio/dev-with-afqmc.txt \
       --device gpu \
       --eval_step 200 \
       --save_step 200 \
       --save_dir /home/aistudio/model \
       --train_batch_size 128 \
       --learning_rate 2E-5 \
       --rdrop_coef 0.0 \
       --seed 2022 \
       --epochs 3 \
       --init_from_ckpt /home/aistudio/model/model_10000/model_state.pdparams

In [ ]:
!python -u \
    predict.py \
    --device gpu \
    --params_path "/home/aistudio/model/model_600/model_state.pdparams" \
    --input_file "/home/aistudio/test.tsv" \
    --batch_size 256 \
    --result_file "/home/aistudio/ccf_qianyan_qm_result_A.csv"

In [ ]:
!pip install pypinyin

In [ ]:
import pandas as pd
import numpy as np
import pypinyin

data = pd.read_csv(filepath_or_buffer='/home/aistudio/test.tsv', sep='	', header=None, names=['sentence1', 'sentence2'])
sentence1 = np.asarray(data.iloc[:, 0:1]).tolist()
sentence2 = np.asarray(data.iloc[:, 1:2]).tolist()

labels = pd.read_csv('/home/aistudio/ccf_qianyan_qm_result_A.csv', header=None, names=['label'])
labels = np.asarray(labels.iloc[:, 0:1]).transpose().tolist()[0]
len(sentence1), len(sentence2), labels

def check_synonyms(s1, s2):
    if len(s1) != len(s2):
        return False
    for i in range(len(s1)):
        if len(s1[i]) == 1 and len(s2[i]) == 1:
            if s1[i][0] != s2[i][0]:
                return False
        elif len(s1[i]) > 1 and len(s2[i]) == 1:
            if not (s2[i][0] in s1[i]):
                return False
        elif len(s2[i]) > 1 and len(s1[i]) == 1:
            if not (s1[i][0] in s2[i]):
                return False
        else:
            flag = False
            for j in range(len(s1[i])):
                if s1[i][j] in s2[i]:
                    flag = True
                    break
            if not flag:
                return False
    return True

count = 0
output = []

for i in range(100000):
    stc1 = str(sentence1[i][0])
    stc2 = str(sentence2[i][0])

    if str(sentence1[i][0]).__contains__('×') or str(sentence2[i][0]).__contains__('×'):
        labels[i] = 0
        output.append(str(stc1) + '\t' + str(stc2) + '\t' + str(labels[i]))
    else:
        s1 = [pypinyin.pinyin(word, heteronym=True, style=pypinyin.STYLE_NORMAL)[0] for word in sentence1[i][0]]
        # s2 = [pypinyin.pinyin(word, heteronym=True, style=pypinyin.STYLE_NORMAL)[0] for word in sentence2[i][0]]

        if len(stc1) == len(stc2):
            for j in range(len(stc1)):
                temp = stc2[j:] + stc2[:j]
                s2 = [pypinyin.pinyin(word, heteronym=True, style=pypinyin.STYLE_NORMAL)[0] for word in temp]
                if check_synonyms(s1, s2):
                    # print(str(stc1) + '\t' + str(stc2) + '\t' + str(results[i][0]))
                    count += 1
                    output.append(str(stc1) + '\t' + str(stc2) + '\t' + str(labels[i]))
                    labels[i] = 1
                    break

    # if check_synonyms(s1, s2):
    #     # print(str(i + 1) + "\t" + sentence1[i][0] + "\t" + sentence2[i][0])
    #     results[i][0] = 1
    #     count += 1

print('测试集同音率：{}%；同音数：{}'.format(str(count / 100000 * 100), count))

for i in range(100000):
    if str(sentence1[i][0]).__contains__('×') or str(sentence2[i][0]).__contains__('×'):
        labels[i] = 0

with open('result/synonyms_list.txt', 'w', encoding='utf8', newline='\n') as f:
    for row in output:
        f.write(row + '\n')

with open(file='result/pinyin_misspelling_result_B.csv', mode='w', newline='\n', encoding='utf8') as f:
    for result in labels:
        f.write(str(result) + '\n')